In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import collections
import os
import random
import numpy as np
from tqdm import tqdm
import sys, email
import pandas as pd
import math
import re


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
#########################################################
# Load Enron dataset
#########################################################
ENRON_EMAIL_DATASET_PATH = 'enron.csv'

# load enron dataset
import pandas as pd

emails_df = pd.read_csv(ENRON_EMAIL_DATASET_PATH)
print(emails_df.shape)
emails_df.head()


(517401, 8)


,Message-ID,Date,From,To,Subject,Message,Cc,Bcc
0,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,NaN,Here is our forecast\n\n \n,NaN,NaN
1,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,Traveling to have a business meeting takes the...,NaN,NaN
2,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,test successful. way to go!!!\n,NaN,NaN
3,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,NaN,"Randy,\n\n Can you send me a schedule of the s...",NaN,NaN
4,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,Let's shoot for Tuesday at 11:45. \n,NaN,NaN


In [3]:
#########################################################
# change wor2vec model to work with Enron emails
#########################################################
emails_sample_df = emails_df.copy()
emails_df.fillna('', inplace = True)

In [4]:
# clean up subject line
emails_sample_df['Subject'] = emails_sample_df['Subject'].str.lower()
emails_sample_df['Subject'] = emails_sample_df['Subject'].str.replace('re:', ' ')
emails_sample_df['Subject'] = emails_sample_df['Subject'].str.replace('fw:', ' ')
emails_sample_df['Subject'] = emails_sample_df['Subject'].str.replace(r'[^a-z]', ' ')
emails_sample_df['Subject'] = emails_sample_df['Subject'].str.replace(r'\s+', ' ')
emails_sample_df['Subject'].fillna('', inplace = True)

# clean up content line
emails_sample_df['Message'] = emails_sample_df['Message'].str.lower()
emails_sample_df['Message'] = emails_sample_df['Message'].str.replace(r'[^a-z]', ' ')
emails_sample_df['Message'] = emails_sample_df['Message'].str.replace('fw:', ' ')
emails_sample_df['Message'] = emails_sample_df['Message'].str.replace(r'\s+', ' ')
emails_sample_df['Message'].fillna('', inplace = True)



C:\Users\Aaron\AppData\Local\Temp/ipykernel_34004/1235119367.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  emails_sample_df['Subject'] = emails_sample_df['Subject'].str.replace(r'[^a-z]', ' ')
C:\Users\Aaron\AppData\Local\Temp/ipykernel_34004/1235119367.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  emails_sample_df['Subject'] = emails_sample_df['Subject'].str.replace(r'\s+', ' ')
C:\Users\Aaron\AppData\Local\Temp/ipykernel_34004/1235119367.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  emails_sample_df['Message'] = emails_sample_df['Message'].str.replace(r'[^a-z]', ' ')
C:\Users\Aaron\AppData\Local\Temp/ipykernel_34004/1235119367.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  emails_sample_df['Message'] = emails_sample_df['Message'].str.replace(r'\s+', ' ')


In [5]:
import string
alphabet = string.ascii_letters+string.punctuation
emails_sample_df['Message'].str.strip(alphabet).astype(bool).any()

True

In [6]:
emails_sample_df["Message"]

0                                     here is our forecast 
1         traveling to have a business meeting takes the...
2                                test successful way to go 
3         randy can you send me a schedule of the salary...
4                               let s shoot for tuesday at 
                                ...                        
517396    this is a trade with oil spec hedge ng john la...
517397    some of my position is with the alberta term b...
517398     original message from doucet dawn sent wednes...
517399    analyst rank stephane brodeur chad clark ian c...
517400    i think the ymca has a class that is for peopl...
Name: Message, Length: 517401, dtype: object

In [7]:
# create sentence list
emails_text = (emails_sample_df["Subject"] + ". " + emails_sample_df["Message"]).tolist()


In [8]:
emails_text

['. here is our forecast ',
 ' . traveling to have a business meeting takes the fun out of the trip especially if you have to prepare a presentation i would suggest holding the business plan meetings here then take a trip without any formal business meetings i would even try and get some honest opinions on whether a trip is even desired or necessary as far as the business meetings i think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not too often the presenter speaks and the others are quiet just waiting for their turn the meetings might be better if held in a round table discussion format my suggestion for where to go is austin play golf and rent a ski boat and jet ski s flying somewhere takes too much time ',
 ' test. test successful way to go ',
 '. randy can you send me a schedule of the salary and level of everyone in the scheduling group plus your thoughts on any changes that need to be made patti s for

In [9]:
sentences = ' '.join([str(i) for i in emails_text])
words = sentences.split()

print('Data size', len(words))


Data size 142969566


In [10]:
# get unique words and map to glove set
print('Unique word count', len(set(words))) 

Unique word count 595623


In [11]:
# drop rare words
vocabulary_size = 50000

In [12]:
def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in tqdm(words):
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)

100%|███████████████████████████████████████████████████████████████| 142969566/142969566 [00:50<00:00, 2840735.07it/s]


In [13]:
del words  
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 2653112], ('the', 5679772), ('to', 4059185), ('and', 2589752), ('of', 2388302)]
Sample data [115, 145, 11, 53, 2020, 115, 4033, 2, 25, 5] ['.', 'here', 'is', 'our', 'forecast', '.', 'traveling', 'to', 'have', 'a']


In [18]:
####################################################################
# find matches with glove 
####################################################################
GLOVE_DATASET_PATH = 'glove.6B.300d.txt'

from contextlib import suppress
from tqdm import tqdm
import string


embeddings_index = {}
f = open(GLOVE_DATASET_PATH, encoding="utf8")
word_counter = 0
for line in tqdm(f):
  values = line.split()
  word = values[0]
  if word in dictionary:
    with suppress(ValueError):
        coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
  word_counter += 1
f.close()




print('Found %s word vectors matching enron data set.' % len(embeddings_index))
print('Total words in GloVe data set: %s' % word_counter)


400000it [00:10, 39721.36it/s]

Found 38274 word vectors matching enron data set.
Total words in GloVe data set: 400000


In [19]:
print(len(embeddings_index))
#38845 word match

38274


In [20]:
### # Check out some clusters
#########################################################
# create a dataframe using the embedded vectors and attach the key word as row header
import pandas as pd
enrond_dataframe = pd.DataFrame(embeddings_index)
enrond_dataframe = pd.DataFrame.transpose(enrond_dataframe)
 
# See what it learns and look at clusters to pull out major themes in the data
CLUSTER_SIZE = 300 
# cluster vector and investigate top groups
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=CLUSTER_SIZE)
cluster_make = kmeans.fit_predict(enrond_dataframe)

labels = kmeans.predict(enrond_dataframe)
import collections
cluster_frequency = collections.Counter(labels)
print(cluster_frequency)
cluster_frequency.most_common()

clusters = {}
n = 0
for item in labels:
    if item in clusters:
        clusters[item].append(list(enrond_dataframe.index)[n])
    else:
        clusters[item] = [list(enrond_dataframe.index)[n]]
    n +=1

for k,v in cluster_frequency.most_common(100):
  print('\n\n')
  print('Cluster:', k)
  print (' '.join(clusters[k]))

Counter({3: 521, 175: 508, 25: 491, 35: 471, 197: 431, 164: 416, 270: 362, 79: 333, 73: 329, 11: 309, 266: 294, 173: 289, 127: 287, 278: 286, 130: 273, 85: 271, 262: 266, 113: 256, 289: 255, 69: 250, 132: 247, 114: 234, 16: 233, 23: 231, 280: 230, 105: 224, 255: 223, 103: 220, 40: 219, 52: 217, 237: 217, 102: 217, 67: 210, 80: 209, 120: 208, 30: 207, 156: 204, 57: 199, 62: 198, 48: 197, 253: 196, 117: 194, 241: 191, 44: 191, 285: 191, 107: 189, 177: 189, 209: 186, 66: 184, 99: 182, 72: 180, 32: 180, 269: 180, 154: 179, 186: 179, 63: 179, 70: 177, 174: 177, 46: 174, 158: 174, 147: 173, 223: 173, 202: 171, 176: 170, 137: 170, 145: 167, 182: 166, 240: 166, 28: 163, 21: 163, 22: 159, 258: 157, 246: 156, 216: 154, 192: 154, 15: 152, 189: 151, 17: 151, 206: 151, 61: 151, 106: 148, 194: 147, 39: 146, 257: 146, 288: 145, 142: 145, 27: 143, 109: 141, 138: 141, 201: 138, 188: 138, 112: 137, 260: 136, 6: 136, 77: 135, 5: 135, 196: 135, 268: 134, 101: 134, 124: 134, 139: 134, 14: 134, 234: 134, 43